In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from timm import create_model
import numpy as np
from sklearn.metrics import f1_score
from torch.optim.lr_scheduler import ReduceLROnPlateau # Import scheduler
import pandas as pd # Import pandas for logging

# ==== Cài đặt chung ====
data_dir = r"C:\Users\Admin\Documents\Python Project\Res conn 2025\final_data\not_seg"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_save_path = 'best_swin_model.pth' # Đường dẫn để lưu mô hình tốt nhất
log_file_path = 'training_log.csv' # Đường dẫn để lưu file log


# ==== Tiền xử lý Dữ liệu ====
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

val_test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load dataset với transform mặc định để chia dataset
full_dataset = datasets.ImageFolder(root=data_dir, transform=val_test_transform)

# ==== Chia dataset thành train, val, test ====
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

total_size = len(full_dataset)
train_size = int(train_ratio * total_size)
val_size = int(val_ratio * total_size)
test_size = total_size - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(full_dataset, [train_size, val_size, test_size])

# Gán transform riêng biệt cho từng tập con
# Lưu ý: Các Subset (train_dataset, val_dataset, test_dataset)
# tham chiếu đến cùng một đối tượng dataset gốc.
# Việc thay đổi transform của 'dataset' trong mỗi Subset sẽ áp dụng
# cho các mẫu được lấy ra từ Subset đó.
train_dataset.dataset.transform = train_transform
val_dataset.dataset.transform = val_test_transform
test_dataset.dataset.transform = val_test_transform


# ==== DataLoader ====
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=os.cpu_count()//2 if os.cpu_count() else 0)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=os.cpu_count()//2 if os.cpu_count() else 0)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=os.cpu_count()//2 if os.cpu_count() else 0)

# ==== Cấu hình Mô hình ViT ====
# Tạo mô hình Swin Transformer với pretrained weights
# num_classes được đặt bằng số lượng lớp trong dataset của bạn
model = create_model('swin_base_patch4_window7_224', pretrained=True, num_classes=len(full_dataset.classes))
model = model.to(device)

# Định nghĩa hàm mất mát và bộ tối ưu hóa
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0.001)

# ==== Learning Rate Scheduler ====
# Giảm Learning Rate khi validation loss không cải thiện sau 'patience' epoch
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

# ==== EarlyStopping ====
class EarlyStopping:
    def __init__(self, patience=4, min_delta=0, path='best_model.pth'):
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = np.inf
        self.counter = 0
        self.path = path # Đường dẫn để lưu mô hình tốt nhất

    def __call__(self, val_loss, model_state_dict): # Thêm đối số model_state_dict
        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
            torch.save(model_state_dict, self.path) # Lưu trạng thái mô hình
            print(f"Validation loss decreased to {val_loss:.4f}. Saving model...")
            return False
        else:
            self.counter += 1
            if self.counter >= self.patience:
                print(f"Early stopping triggered! Validation loss has not improved for {self.patience} epochs.")
                return True
            return False

early_stopping = EarlyStopping(patience=5, min_delta=0.001, path=model_save_path)

# ==== Training Loop ====
epochs = 300
log_data = [] # Danh sách để lưu dữ liệu log

print("Starting training...")
for epoch in range(epochs):
    # --- Giai đoạn Huấn luyện ---
    model.train() # Đặt mô hình về chế độ huấn luyện
    running_loss = 0.0
    for batch_idx, (imgs, labels) in enumerate(train_loader):
        imgs, labels = imgs.to(device), labels.to(device)
        
        optimizer.zero_grad() # Đặt gradient về 0
        outputs = model(imgs) # Lan truyền tiến
        loss = criterion(outputs, labels) # Tính toán mất mát
        loss.backward() # Lan truyền ngược
        optimizer.step() # Cập nhật trọng số
        
        running_loss += loss.item() * imgs.size(0) # Cộng dồn loss (nhân với batch size để có tổng loss)
    
    train_loss = running_loss / len(train_loader.dataset) # Loss trung bình trên tập huấn luyện

    # --- Giai đoạn Đánh giá trên tập Validation ---
    model.eval() # Đặt mô hình về chế độ đánh giá
    val_loss = 0.0
    with torch.no_grad(): # Tắt tính toán gradient trong giai đoạn này
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * imgs.size(0)
    
    val_loss /= len(val_loader.dataset) # Loss trung bình trên tập validation

    # Cập nhật Learning Rate Scheduler
    scheduler.step(val_loss)

    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, LR: {optimizer.param_groups[0]['lr']:.6f}")

    # Ghi log dữ liệu
    log_data.append({
        'epoch': epoch + 1,
        'train_loss': train_loss,
        'val_loss': val_loss,
        'learning_rate': optimizer.param_groups[0]['lr']
    })

    # Kiểm tra Early Stopping
    if early_stopping(val_loss, model.state_dict()): # Truyền trạng thái mô hình
        print(f"Early stopping at epoch {epoch+1}")
        break

# Lưu log quá trình huấn luyện vào file CSV
df_log = pd.DataFrame(log_data)
df_log.to_csv(log_file_path, index=False)
print(f"Training log saved to {log_file_path}")


print("\nLoading best model for testing...")
model.load_state_dict(torch.load(model_save_path))
model.eval()

all_labels = []
all_predictions = []
correct = 0
total = 0

print("Starting evaluation on test set...")
with torch.no_grad():
    for imgs, labels in test_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model(imgs)
        _, predicted = torch.max(outputs, 1)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

accuracy = 100 * correct / total
f1 = f1_score(all_labels, all_predictions, average='weighted')

print(f"\n===== Test Results =====")
print(f"Accuracy on test set: {accuracy:.2f}%")
print(f"F1 Score on test set: {f1:.4f}")
print("========================")

c:\Users\Admin\anaconda3\envs\tf\lib\site-packages\torch\optim\lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Starting training...
Epoch 1/300, Train Loss: 0.3152, Val Loss: 0.1805, LR: 0.000100
Validation loss decreased to 0.1805. Saving model...
Epoch 2/300, Train Loss: 0.1694, Val Loss: 0.1781, LR: 0.000100
Validation loss decreased to 0.1781. Saving model...
Epoch 3/300, Train Loss: 0.1607, Val Loss: 0.1831, LR: 0.000100
Epoch 4/300, Train Loss: 0.1305, Val Loss: 0.1652, LR: 0.000100
Validation loss decreased to 0.1652. Saving model...
Epoch 5/300, Train Loss: 0.1110, Val Loss: 0.1839, LR: 0.000100
Epoch 6/300, Train Loss: 0.0990, Val Loss: 0.2065, LR: 0.000100
Epoch 7/300, Train Loss: 0.0827, Val Loss: 0.2048, LR: 0.000100
Epoch 8/300, Train Loss: 0.0852, Val Loss: 0.2141, LR: 0.000010
Epoch 9/300, Train Loss: 0.0499, Val Loss: 0.2231, LR: 0.000010
Early stopping triggered! Validation loss has not improved for 5 epochs.
Early stopping at epoch 9
Training log saved to training_log.csv

Loading best model for testing...
Starting evaluation on test set...


C:\Users\Admin\AppData\Local\Temp\ipykernel_9872\1843193495.py:162: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_save_path))



===== Test Results =====
Accuracy on test set: 92.13%
F1 Score on test set: 0.9188
